In [14]:
#set up our imports 
import pandas as pd

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson
#load some data into a dataframe
wd = [31, 37, 9, 9, 9, 9, 6, 6, 104]
cols = ['point', 'prov', 'latg', 'latm', 'lngg', 'lngm', 'h', 'number', 'oaci']

dfp = pd.read_fwf('estaciones_smn.txt', widths=wd, names=cols)

dfp = dfp[2:]
dp = {
    'point': str, 'prov': str, 
    'latg': int, 'latm': int, 'lngg': int, 'lngm': int, 
    'h': int, 'number': int,
    'oaci': str
}

for k in dp:
    dfp[k] = dfp[k].astype(dp[k])

dfp['lat'] = dfp['latg'] - (dfp['latm'] / 60)
dfp['lng'] = dfp['lngg'] - (dfp['lngm'] / 60)


v = [8, 6, 6, 101-8-12]
dft = pd.read_fwf("temps.txt", widths=v, names=['date', 'max', 'min', 'loc'])
dft = dft[3:]

dft['ansidate'] = pd.to_datetime(dft['date'], format='%d%m%Y')
del(dft['date'])
dt = {
    'max': float, 'min': float, 
    'ansidate': 'datetime64[ns]',
    'loc': str
}

for k in dt:
    dft[k] = dft[k].astype(dt[k])

dft = dft.iloc[::-1]

geojson_markers = df_to_geojson(dfp[['point', 'lat', 'lng', 'h']], {'point', 'h'}, lat='lat', lon='lng')
#let's look at the dataframe head
dfp.head()


,point,prov,latg,latm,lngg,lngm,h,number,oaci,lat,lng
2,BASE BELGRANO II,ANTARTIDA,-77,52,-34,34,256,89034,SAYB,-77.866667,-34.566667
3,BASE CARLINI (EX JUBANY),ANTARTIDA,-62,14,-58,38,11,89053,SAYJ,-62.233333,-58.633333
4,BASE ESPERANZA,ANTARTIDA,-63,24,-56,59,24,88963,SAYE,-63.400000,-56.983333
5,BASE MARAMBIO,ANTARTIDA,-64,14,-56,43,198,89055,SAWB,-64.233333,-56.716667
6,BASE ORCADAS,ANTARTIDA,-60,45,-44,43,12,88968,SAYO,-60.750000,-44.716667


In [5]:
dft.head()

,max,min,loc,ansidate
44615,15.1,-1.8,VILLA REYNOLDS AERO,2018-07-15
44614,20.0,9.7,VILLA MARIA DEL RIO SECO,2018-07-15
44613,12.0,2.5,VILLA GESELL AERO,2018-07-15
44612,21.2,4.4,VILLA DOLORES AERO,2018-07-15
44611,9.6,3.1,VIEDMA AERO,2018-07-15


In [20]:
%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from ipywidgets import HBox, widgets
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    LayersControl, Marker, MarkerCluster,
    GeoJSON, FullScreenControl, WidgetControl
)

today = datetime.today().date()
out1 = widgets.Output()
from_date = widgets.DatePicker(description='From', disabled=False)
to_date = widgets.DatePicker(description='To', disabled=False)
dates = HBox([from_date, to_date])

def clear_plt(*args, **kwargs):
    plt.cla()
    plt.clf()
    plt.close()
    out1.clear_output()


def mark_click(*args, **kwargs):
    global out1
    if kwargs.get("event", "other") == "click":
        props = kwargs.get("properties", {})
        point = props.get("point", None)
        if point is not None:
            data = dft[dft["loc"].str.startswith(point)]
            frmd = from_date.value or datetime(2010, 1, 1)
            tod = to_date.value or today
            data = data[data.ansidate.between(frmd, tod)]
            ax = data.plot(x='ansidate', y=['min', 'max'])
            ax.set(xlabel="Date", ylabel="Temp")
            plt.xticks(rotation=90)
            with out1:
                plt.title(f"{point} - Alt.: {props.get('h', 'No Data')}")
                plt.show()


geo_json = GeoJSON(
    data=geojson_markers,
    name="Datos del SMN",
    style={'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2}
)
geo_json.on_click(mark_click)

btn = widgets.Button(description="Clear")
btn.on_click(clear_plt)

m = Map(center=(dfp.lat.mean(), dfp.lng.mean()), zoom=6)
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, (today - timedelta(days=1)).isoformat())
    
m.add_layer(nasa_layer)
m.add_layer(geo_json)

m.add_control(WidgetControl(widget=dates, position="bottomleft"))
m.add_control(WidgetControl(widget=btn, position='bottomright'))
m.add_control(WidgetControl(widget=out1, position='bottomright'))
m.add_control(FullScreenControl())
m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

/home/manuel/.virtualenvs/temps/lib/python3.7/site-packages/pandas/core/series.py:4580: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  rmask = self <= right
